# Global Terrorism Analysis

import required libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import datetime as dt 
import matplotlib.pyplot as plt
from dash import Dash, dcc, html,Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
from dash.dependencies import Input,Output
import random
import textwrap
import datetime as dt 
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

loading data

In [ ]:
path='E:\ITIAI2021\Python\goterro.csv'

df=pd.read_csv(path, encoding='latin-1')

information about data to see if it contains Null values or columns that is not necessary in our DataFrame

In [ ]:
#df.info

In [ ]:
#df.columns.tolist()

In [ ]:
#df['target1']

Select specific features that we need in our Exploration

In [ ]:
data=['iyear','country_txt','region','region_txt','provstate','city','latitude','longitude','specificity','vicinity','country','nkill','nwound','success','attacktype1_txt','targtype1_txt','target1','gname']

In [ ]:
Terr=pd.DataFrame(df[data])
#Terr

Information about Data after selection

In [ ]:
Terr.info()

In [ ]:
null=Terr.isnull().sum()
#null

In [ ]:
Terr.dropna(subset=['city'],inplace=True)

Most affected destinations

In [ ]:
attack=Terr['targtype1_txt'].unique()
TargetAttack=Terr['targtype1_txt'].value_counts().sort_values(ascending=True)


In [ ]:
#pio.templates
fig = go.Figure()
fig.add_trace(go.Bar(x=attack,y=TargetAttack.sort_values(ascending=False),
                base=0,
                orientation='v',
                marker_color='crimson',
                name='kill/attackType'
                    
                     
                ))
fig.update_layout(template='plotly_dark',
    title="Most Affected Destinations",              
    title_x=0.5,              
    title_y=0.9,
    xaxis_title="Attack destinations",
    yaxis_title="Number of Attacks",
    legend_title="Legend Title")

fig.show()

Number of dead and wounded people result from each attack around world

In [ ]:
kills=['nkill','nwound']
for k in kills:
    Terr[k] = Terr[k].transform(lambda x: x.fillna(x.mean()))

In [ ]:
kill_attackType=Terr[['attacktype1_txt','nwound','nkill']].groupby(["attacktype1_txt"],axis=0).sum()
kill_attackType.sort_values(by=['nwound'],inplace=True)
kill_attackType['wound+nkill']=kill_attackType['nwound']+kill_attackType['nkill']


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=kill_attackType.index,y=kill_attackType['wound+nkill'].sort_values(ascending=True),
                base=0,
                marker_color='crimson',
                name='kill/attackType',orientation='v'
           
                ))

fig.update_layout(template='plotly_dark',
    title="Total Numer of dead and wounded result from each attack",
    title_x=0.5,              
    title_y=0.9,
    xaxis_title="AttackType",
    yaxis_title="Total num of dead&wounded people",
                  
    )
fig.show()

Total number of attacks that happened in each region/year

In [ ]:
c=pd.crosstab(Terr.iyear,Terr.region_txt)
a=c.index
f=c.columns


In [ ]:
area_fig = px.area(c,x=a,y=f,title='Number of attacks happened in each region/Year',template='plotly_dark')
area_fig.update_xaxes(title_text='Year')
area_fig.update_yaxes(title_text='Number of attacks')
area_fig.update_layout(
    title_x=0.5,
    legend=dict(
        x=0.04,  # value must be between 0 to 1.
        y=1.1,   # value must be between 0 to 1.
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
           
        ),
    )
)
fig.update_layout(legend={'itemsizing': 'constant'})
area_fig.show()

# Dashboard 

In [ ]:
app=Dash(external_stylesheets=(['https://codepen.io/chriddyp/pen/bWLwgP.css',dbc.themes.DARKLY]))
app.layout = html.Div([
    html.Br(),html.Br(),
    html.H3('Global Terrorism',style={'color':'red','textAlign':'center','marginTop':5,'fontSize':60,'border':3}),
    html.Hr(),
    
    dbc.Card([
        dbc.CardBody([
            html.H4(className="card-title",id="country_year",style={'fontSize':25}),
            html.P(className="card-text",id="card_text2"),
        ])
    ],
        style={'display': 'inline-block',
               'width': '12.5%',
               'text-align': 'center',
               'color':'#00C5CD',
               'margin-left':'42%',
               'margin-right':'25%',
               'margin-bottom':'1%',
               'backgroundColor':'dark',
               'font-weight': 'bold',
               'font-family': 'Arial'
              },
        outline=True),
      dbc.Row(
                [ dbc.Col( dbc.Card(
                 dbc.CardBody(
                    [
                        html.H5("Attacks", className="card-title",style={'fontSize': 18,"color": "black",'font-weight': 'bold'}),
                        html.P(id='nAttack',style={"color": "black",'font-weight': 'bold','fontSize': 20}),
            
                            ]
                        )
                    ,color="warning")),

                 dbc.Col(dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Death", className="card-title", style={'fontSize': 18,"color": "black",'font-weight': 'bold'}),
                                html.P(id="kill" , style={"color": "black",'font-weight': 'bold','fontSize': 20}),
                            ]
                   
                        )
                         , color="danger")),
                    dbc.Col(dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5("Injured", className="card-title", style={'fontSize': 18,"color": "black",'font-weight': 'bold'}),
                                html.P(id="nwound", style={"color": "black",'font-weight': 'bold','fontSize': 20}
                                    
                                ),
                            ]
                        )
                    , color="success"))
                        ]
                       
    ,style={'textAlign':'center','margin-left':'10%','margin-right': '10%','fontSize':70,'border':3}),
    html.Br(), html.Br(),
   dcc.Graph(id='map_world', config={'displayModeBar': False},style={'margin-left':'10%','margin-right': '10%'}),
    html.Div([
    dcc.Dropdown(id='countries',
                 multi=True,
                 value=['Iraq'],
                 placeholder='Select Countries',
                 options=[{'label': c, 'value': c}
                          for c in sorted(Terr['country_txt'].unique())],style={'backgroundColor': 'black', 'color': 'black'})
        
],style={'margin-left':'25%','width': '50%','backgroundColor': 'black', 'color': 'white','marginTop':'2%'}),
     html.Div([
        dcc.RangeSlider(id='years',
                        min=1970,
                        max=2016,
                        dots=False,
                        value=[2010, 2016],
                        marks={str(yr): str(yr) for yr in range(1970, 2017,5)}),
        ], style={'margin-left':'25%','width': '50%','display': 'inline-block','className':'justify-content-center','background-color': '#000000','marginTop':'2%'}),      
    html.Br(), html.Br(),  
    html.Content('The most affected destinations & Top countries affected by attacks', style={'margin-left': '37%', 'font-size': 25}),
    html.Br(), html.Br(),
    html.Div([
        html.Div([
            dcc.Graph(id='targetAttack',
                      figure={'layout': {'margin': {'r': 10, 't': 50}}},
                      config={'displayModeBar': False})
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            dcc.Graph(id='top_countries_deaths',
                      config={'displayModeBar': False},
                      figure={'layout': {'margin': {'l': 10, 't': 50}}})

        ], style={'width': '48%', 'display': 'inline-block', 'float': 'right'})
    ]),
      html.Content('The most affected Regions&the most dangerous terrorist groups', style={'margin-left': '35%', 'font-size': 20}),
      html.Br(), html.Br(),
   html.Div([
        html.Div([
            dcc.Graph(id='attack_region_year',
                      figure=area_fig,
                      config={'displayModeBar': False})
        ], style={'width': '50%', 'display': 'inline-block'}),
        
        html.Div([
            dcc.Graph(id='attack_gname',
                      config={'displayModeBar': False},
                      figure={'layout': {'margin': {'l': 10, 't': 50}}})

        ], style={'width': '48%', 'display': 'inline-block', 'float': 'right'})
    ]),
    
   
], style={'background-color': '#000000', 'font-family': 'Palatino'})

Summarizing in  BAN's:

1-)Selected country from the drop down list.

2-)Number of attacks that happened in this country.

3-)Number of dead people in that country.

4-)Number of wounded in that country

In [ ]:
@app.callback(
    Output('country_year','children'),
    Input('countries','value'),

    
)
def update_cards_main(country_select):
    
      while country_select :
             c1=country_select.pop()
             output_1= "".join(c1)
             return output_1


@app.callback(
 Output('nAttack','children'),
 Input('countries','value'),
)
def attack_card(country):
    while country:
        c=country.pop()
        df_top_countries=Terr[Terr['country_txt']==c]
        attack=df_top_countries['country_txt'].value_counts()
        attack_country=attack.values[0]
        output_a="".join(str(attack_country).split('.')[0]) 
        return output_a

  

@app.callback(
 Output('kill','children'),
 Input('countries','value'),
)
def kill_card(country_1):
    
        #Co=Terr['country_txt'].unique().tolist()
        while country_1:
             c=country_1.pop()
             df_top_countries=Terr[Terr['country_txt']==c]
             kill_country=df_top_countries.groupby(["country_txt"])['nkill'].sum()
             kill=kill_country.values[0]
             output_k="".join(str(kill).split('.')[0]) 
             return output_k 
            
@app.callback(
 Output('nwound','children'),
 Input('countries','value'),
)
def wound_card(country_2):

       while country_2:
             c=country_2.pop()
             df_top_countries=Terr[Terr['country_txt']==c]
             wound_country=df_top_countries.groupby(["country_txt"])['nwound'].sum()
             wound=wound_country.values[0]
             output_w="".join(str(wound).split('.')[0]) 
             return output_w

      
      
       

Show the distribution of countries on a Map 

In [ ]:
@app.callback(Output('map_world', 'figure'),
             [Input('countries','value'), Input('years', 'value')])
def countries_on_map(countries,years):
        token="pk.eyJ1IjoiMWF5YWVsc2F3eSIsImEiOiJjbDJqajdrYjAwZnd2M2tzMzVlbWs2a2FuIn0.-kxYRdvm2y-8KOXgo1uUsQ"
        df =Terr[Terr['country_txt'].isin(countries)&Terr['iyear'].between(years[0], years[1])]
        fig = fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="country_txt",
                        color_discrete_sequence=["red"], zoom=1)
        fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.show()

        return fig



The most affected destinations result from  number of attacks on it.

In [ ]:
@app.callback(Output('targetAttack', 'figure'),
             [Input('years', 'value')])
def target_Attacks(years):
    df_top_countries = Terr[Terr['iyear'].between(years[0], years[1])]
    attack=df_top_countries['targtype1_txt'].value_counts().sort_values(ascending=False)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=attack.head(10).index.str[:15],y=attack.head(10).values,
                base=0,
                orientation='v',
                marker_color='crimson',
                name='Target Attack'       
                ))
    fig.update_layout(template='plotly_dark',
                     title="Most Affected Destinations",              
                     title_x=0.5,              
                     title_y=0.9,
                     xaxis_title="Attack destinations",
                     yaxis_title="Number of Attacks")
    

    return  fig

Top ten countries with it's number of dead people & wounded around the world

In [ ]:
@app.callback(Output('top_countries_deaths', 'figure'),
             Input('years', 'value'))
def top_countries_kill_wound(years):
    df_top_countries = Terr[Terr['iyear'].between(years[0], years[1])]
    kill_country=df_top_countries.groupby(["country_txt"],axis=0)['nkill'].sum()
    kill_country=kill_country.sort_values(ascending=False)
    kill_country=kill_country.head(10)
    ########wound#######
    wound_country=df_top_countries.groupby(["country_txt"],axis=0)['nwound'].sum()
    wound_country=wound_country.sort_values(ascending=False)
    wound_country=wound_country.head(10)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=kill_country.head(10).index,
                         y=kill_country.head(10).values,
                         marker_color='crimson',
                         orientation='v',
                         name='kill'))
    fig.add_trace(go.Bar(x=wound_country.head(10).index,
                         y=wound_country.head(10).values,
                         marker_color='lightslategrey',
                         orientation='v',
                        name='wound'))
    fig.update_layout(template='plotly_dark',
                     title="Top ten countries with it's number of dead people & wounded around the world",
                     title_x=0.5,              
                     title_y=0.9,
                     yaxis_title="Numof dead people&wounded /each country",
                     xaxis_title="Country",)
    return fig





The most dangerous terrorist groups 

In [ ]:
@app.callback(Output('attack_gname','figure'),
              Input('countries','value'))
              #Input('years','value'))
def gname_Attacks(country):
   while country:
        c=country.pop()
        df_top_countries=Terr[Terr['country_txt']==c]
        df_top_countries=df_top_countries[['gname','nkill']].groupby(['gname'],axis=0).sum().drop('Unknown').sort_values('nkill',ascending=False).head(10)
        df_top_countries=df_top_countries.reset_index()
        fig = go.Figure()
        fig.add_trace(go.Bar(x=df_top_countries['gname'].str[:15],y=df_top_countries['nkill'],
                    base=0,
                    orientation='v',
                    marker_color='crimson',
                    name='Target Attack',
                    hovertext=c
    
            
                    ))
        fig.update_layout(template='plotly_dark',
                         title="The most dangerous terrorist groups ",
                         title_x=0.5,              
                         title_y=0.9,
                         yaxis_title="Group name",
                         xaxis_title="Number of attacks",)

        return  fig

# show dashboard

In [ ]:
app.run_server()

# Number of attack&kills for each country

In [ ]:
"""nAttack_country=Terr['country_txt'].value_counts()
kill_country=Terr[['country_txt','nwound','nkill']].groupby(["country_txt"],axis=0).sum()
kill_country=kill_country.sort_values(by=['nkill'],ascending=False)
top_ten_kw_country=kill_country.head(10)
top_ten_kw_country['wound+nkill']=top_ten_kw_country['nwound']+top_ten_kw_country['nkill']
top_ten_kw_country['Nattack']=nAttack_country
top_ten_kw_country=top_ten_kw_country.reset_index()
Terr['Country_txt'].iloc['Iraq']"""

In [ ]:
"""f=Terr.groupby(["country_txt"],axis=0)['nkill'].sum()
kill_country=f.sort_values(ascending=False)
top_ten_kw_country=kill_country.head(10)
v=top_ten_kw_country.head().values
#kill_country.values
"""

In [ ]:
"""fig = go.Figure()
fig.add_trace(go.Bar(x=top_ten_kw_country.index,y=v,
                base=[-500,-600,-700],
                marker_color='crimson',
                orientation='v',
                name='kill&wound'))

fig.update_layout(template='plotly_dark',
    title="Total Number of Kills and wound for each country.",
    yaxis_title="Nkill+Nwound &Nattack",
    xaxis_title="Country",
    )

fig.show()"""

9# top ten attacks group

In [ ]:
#df1=Terr[['gname','nkill','nwound']].groupby(['gname'],axis=0).sum().drop('Unknown').reset_index().sort_values('nkill',ascending=False).head(15)

In [ ]:
"""df1=Terr[['gname','country_txt','iyear']].groupby(['country_txt'],axis=0).sum().reset_index().sort_values('nkill',ascending=False).head(15)
#df_top_countries = df1[df1['iyear'].between(years[0], years[1])]
attack=df1['gname']
fig = go.Figure()
fig.add_trace(go.Bar(x=attack.head(10).values,y=attack.head(10).sort_index(ascending=False).index,
            base=0,
            orientation='v',
            marker_color='crimson',
            name='Terrorist group'       
            ))
fig.update_layout(template='plotly_dark',
                 title="The most dangerous terrorist groups",
                 yaxis_title="Nattacks",
                 xaxis_title="Terrorist groups",)
fig.show()"""

In [ ]:
#df1=Terr[['gname','country_txt']].groupby(['gname'],axis=0).sum().drop('Unknown').reset_index().sort_values('nkill',ascending=False).head(15)

In [ ]:
#attack.sort_index(ascending=False)

In [ ]:
#df1['gname'].str.replace()

In [ ]:
"""y=df1['nkill'].to_list()
yng=[]
for i in y:
    ne=-1*i
    yng.append(ne)
#yng """   

In [ ]:
"""kill_region=Terr[['region_txt','country','city','nkill','nwound',"latitude","longitude"]].groupby(["region_txt"],axis=0).sum().reset_index()
kill_region=kill_region.sort_values(by=['nkill'])
# kill_region=pd.concat([kill_region,Terr['latitude'],Terr['longitude']],axis=1)
# kill_region
kill_region"""

In [ ]:
"""fig = px.scatter_mapbox(Terr,lat="latitude",lon="longitude",hover_data=['nkill','country_txt'],animation_frame='iyear',
                        color_discrete_sequence=["fuchsia"], zoom=.2, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()"""

In [ ]:
"""fig = px.scatter_mapbox(kill_region,lat="latitude", lon="longitude", hover_name='country',height=600)
#token = open(".mapbox_token").read()
fig.update_layout(mapbox_style="dark")
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()"""